## Enhanced Lee Filter Recipe

##### The **Enhanced Lee Filter (ELF)** is presented as an **adaptive filter**, meaning that for calculating the new value for each pixel in our image, we will use the standard deviation obtained from the values of all pixels within a defined window (e.g 5x5, 7x7, etc.). The old pixel value is being replaced by the new filtered value. Adaptive filters have the advantage of supressing noise while preserving the sharpness and detail of features within an image. This is also the case for ELF, which preserves the texture of the image and minimizes the radiometric loss, while removing the noise. ELF is an alteration of the classical Lee filter, and uses the same principle of local statistics (mean and variance), namely the use of the **coefficient of variation** calculated in each window. An aproximation of the 'grey level' for each pixel is done by computing the weighted sum of the center pixel value, the mean value and the variance in the kernel. Pixels are then assigned one of the three classes:

 - Homogenous: the new pixel value is the value of the average within the filter window;
 - Heterogenous: the new pixel value is the value of the weighted average;
 - Point target: the new pixel value is the same as the old pixel value. 
 
The original implementation of the Enhanced Lee Filter is described in Lopez, A., Touzi, R., Nezry, E., 1990, Adaptive Speckle Filters and Scene Heterogeneity, IEEE Transactions on Geoscience and Remote Sensing, Vol. 28, No.6

In [1]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import math
from skimage import exposure
import os
import glob

In [4]:
def elf_filter(input_file, output_file):
    print('Begin filtering file ' + input_file)
    
    # inputing the file
    sar_image=gdal.Open(input_file)
    sar_band=sar_image.GetRasterBand(1)
    sar_band.GetMetadata()
    img_array=sar_band.ReadAsArray()
    img_array = img_array.astype(np.float32)
    [cols, rows]=img_array.shape

    
    #setup requirements for Enhanced Lee Filter
    
    window_size = 5  
    nlooks= 4.4 #default value for Sentinel-1 senzor, can be modified
    cu = 1 / math.sqrt(nlooks)
    cmax = math.sqrt(1 + 2 / nlooks)
    damp = 1.0
    
    mean=cv.blur(img_array, (window_size, window_size))                                                
    mean2 = cv.blur(img_array*img_array, (window_size, window_size))
    stddev = cv.sqrt(mean2-mean*mean)
    stddev = np.nan_to_num(stddev)
    ci= cv.divide(stddev, mean)
    
    out = np.zeros(img_array.shape, dtype=np.float32)
    
    # define filter function
    
    def enhanced_lee():
        for i in range(cols):
            for j in range(rows):
                if ci[i,j]<=cu:
                    out[i,j]= mean[i,j]         
                elif ci[i,j]>=cmax: 
                    out[i,j]=img_array[i,j]
                else:
                    center_pixel=img_array[i,j]
                    weight= np.exp((-(damp)*(ci[i,j]-cu))/(cmax-ci[i,j]))
                    out[i,j]=mean[i,j]*weight+center_pixel*(1-weight)
        return out

    #filter image
    
    filtered_image=enhanced_lee()
    
    # writing output file
    driver = gdal.GetDriverByName("GTiff")
    output = driver.Create(output_file, rows, cols, 1, gdal.GDT_Float32)
    proj = output.SetGeoTransform(sar_image.GetGeoTransform())  
    output.SetProjection(sar_image.GetProjection())
    output.SetGCPs(sar_image.GetGCPs(), "4326")
    geoband = output.GetRasterBand(1)
    geoband.WriteArray(filtered_image)
    output.GetRasterBand(1).SetNoDataValue(-9999)
    output.FlushCache()
    output = None
    band=None
    print('Created file ' + output_file)
    
    print('End filtering file ' + input_file)

In [7]:
for input_file in glob.glob('/home/cristina/Seeps/Workflow/Filters/ROI/*/8bit/*tif'):
    output_file = input_file.split(os.sep)
    output_file[-1] = output_file[-1][:-4] + '_ELF_5.tif'
    output_file = output_file[:-2] + ['Filters'] + ['Enhanced_Lee'] + output_file[-2:]
    output_file = os.sep.join(output_file)
    elf_filter(input_file, output_file)

Begin filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI1.tif
Created file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/Filters/Enhanced_Lee/8bit/ROI1_ELF_5.tif
End filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI1.tif
Begin filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI3.tif
Created file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/Filters/Enhanced_Lee/8bit/ROI3_ELF_5.tif
End filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI3.tif
Begin filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI4.tif
Created file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/Filters/Enhanced_Lee/8bit/ROI4_ELF_5.tif
End filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI4.tif
Begin filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI2.tif
Created file /home/